In [5]:
csv_file_path = "./content/try3.csv"

In [6]:
!git clone https://github.com/ztuntrade/untrade-sdk.git
!pip install ./untrade-sdk/.

import uuid
import os
from untrade.client import Client

fatal: destination path 'untrade-sdk' already exists and is not an empty directory.


Processing c:\users\entro\zelta\untrade-sdk
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for untrade: filename=untrade-0.1.0-py3-none-any.whl size=5100 sha256=0a784c9993bca0ac451fd772781cbddeefed2f12d4d0a941edecc7aa6c166c83
  Stored in directory: c:\users\entro\appdata\local\pip\cache\wheels\a5\e7\47\f7aaac7475c7ed89d1026a37e46f91dc89288148b8e6b287f1
Successfully built untrade
  Attempting uninstall: untrade
    Found existing installation: untrade 0.1.0
    Uninstalling untrade-0.1.0:
      Successfully uninstalled untrade-0.1.0


In [7]:
def perform_backtest(csv_file_path):
     client = Client()
     result = client.backtest(
         jupyter_id="test",
         file_path=csv_file_path,
         leverage=1,
     )
     return result


def perform_backtest_large_csv(csv_file_path):
    client = Client()
    file_id = str(uuid.uuid4())
    chunk_size = 90 * 1024 * 1024  # 90 MB
    total_size = os.path.getsize(csv_file_path)
    total_chunks = (total_size + chunk_size - 1) // chunk_size
    chunk_number = 0

    # Handle small files
    if total_size <= chunk_size:
        total_chunks = 1
        result = client.backtest(
            file_path=csv_file_path,
            leverage=1,
            jupyter_id="test",
        )
        for value in result:
            print(value)
        return result

    # Process large files in chunks
    with open(csv_file_path, "rb") as f:
        while True:
            chunk_data = f.read(chunk_size)
            if not chunk_data:
                break

            # Save each chunk temporarily in /tmp
            chunk_file_path = f"/tmp/{file_id}_chunk{chunk_number}.csv"
            with open(chunk_file_path, "wb") as chunk_file:
                chunk_file.write(chunk_data)

            # Perform backtest on the current chunk
            result = client.backtest(
                file_path=chunk_file_path,
                leverage=1,
                jupyter_id="test",
                file_id=file_id,
                chunk_number=chunk_number,
                total_chunks=total_chunks,
            )

            # Process the results of the backtest
            for value in result:
                print(value)

            # Remove the temporary chunk file
            os.remove(chunk_file_path)

            # Move to the next chunk
            chunk_number += 1

    return result

In [9]:
backtest_result = perform_backtest(csv_file_path)
print(backtest_result)
for value in backtest_result:
    print(value)

<generator object Client._handle_response_stream at 0x00000261F02CC380>
data: {
  "jupyter_id": "test",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-09-08 00:00:00",
      "Total Trades": 163,
      "Leverage Applied": 1.0,
      "Winning Trades": 54,
      "Losing Trades": 109,
      "No. of Long Trades": 81,
      "No. of Short Trades": 82,
      "Benchmark Return(%)": 325.632937,
      "Benchmark Return(on $1000)": 3256.329373,
      "Win Rate": 33.128834,
      "Winning Streak": 4,
      "Losing Streak": 7,
      "Gross Profit": 2340.851388,
      "Net Profit": 2096.351388,
      "Average Profit": 12.861051,
      "Maximum Drawdown(%)": 25.979837,
      "Average Drawdown(%)": 6.536448,
      "Largest Win": 544.058835,
      "Average Win": 112.001077,
      "Largest Loss": -138.619328,
      "Average Loss": -36.25419,
      "Maximum Holding Time": "68 days 0:0:0",
      "Average Holding Time": "9 days 15:27: